In [0]:
%sql
USE mc_labs;

##  Calcular o fare_amount (valor da corrida) da viagem anterior para cada pickup_zip. Isso pode ajudar a identificar tendências de preços ou anomalias em uma determinada área de partida.

In [0]:
%sql
-- Exemplo 1: Fare amount da viagem anterior para cada pickup_zip
SELECT
  pickup_zip,
  tpep_pickup_datetime,
  fare_amount,
  LAG(fare_amount, 1, 0.0) OVER (PARTITION BY pickup_zip ORDER BY tpep_pickup_datetime) AS previous_fare_amount
FROM
  trips_raw
WHERE
  pickup_zip IS NOT NULL -- Garante que estamos olhando para zips válidos
ORDER BY
  pickup_zip,
  tpep_pickup_datetime
LIMIT 20;

/*PARTITION BY pickup_zip: Divide os dados em grupos, um para cada código postal de partida. A função LAG operará independentemente dentro de cada grupo.
ORDER BY tpep_pickup_datetime: Dentro de cada grupo de pickup_zip, as linhas são ordenadas pelo tempo de início da viagem.
LAG(fare_amount, 1, 0.0): Pega o fare_amount da linha imediatamente anterior (offset = 1). Se não houver linha anterior (primeira viagem para aquele pickup_zip), retorna 0.0.*/

## Calcular o tempo de início da próxima viagem para cada pickup_zip. Isso pode ser usado para determinar o tempo de inatividade entre as viagens que partem de uma mesma área.

In [0]:
%sql
SELECT
  pickup_zip,
  tpep_pickup_datetime,
  tpep_dropoff_datetime,
  LEAD(tpep_pickup_datetime, 1) OVER (PARTITION BY pickup_zip ORDER BY tpep_pickup_datetime) AS next_trip_pickup_datetime,
  -- Calcular o tempo de inatividade entre o fim da viagem atual e o início da próxima (em minutos)
  (UNIX_TIMESTAMP(LEAD(tpep_pickup_datetime, 1) OVER (PARTITION BY pickup_zip ORDER BY tpep_pickup_datetime)) - UNIX_TIMESTAMP(tpep_dropoff_datetime)) / 60 AS minutes_idle_after_trip
FROM
  trips_raw
WHERE
  pickup_zip IS NOT NULL
ORDER BY
  pickup_zip,
  tpep_pickup_datetime
LIMIT 20;

/*Similar ao LAG, mas LEAD olha para a frente.
Calculamos minutes_idle_after_trip para mostrar uma aplicação prática de LEAD para análise de tempo entre eventos.*/

## Classificar as viagens com os maiores fare_amount por pickup_zip.

In [0]:
%sql
SELECT
  pickup_zip,
  tpep_pickup_datetime,
  fare_amount,
  RANK() OVER (PARTITION BY pickup_zip ORDER BY fare_amount DESC) AS rank_by_fare_amount
FROM
  trips_raw
WHERE
  pickup_zip IS NOT NULL
ORDER BY
  pickup_zip,
  rank_by_fare_amount
LIMIT 30;

## Classificar os códigos postais por população dentro de cada estado (usando sua tabela zip_code_info).

In [0]:
%sql
SELECT
city_name,
zip,
DENSE_RANK() OVER (PARTITION BY city_name ORDER BY zip ASC) AS dense_rank_zip_in_city
FROM
zip_code_info
WHERE
city_name IS NOT NULL -- Garante que estamos olhando para cidades válidas
ORDER BY
city_name,
dense_rank_zip_in_city
LIMIT 30;

## Dividir todas as viagens em 4 quartis com base no fare_amount. Isso pode ser útil para segmentar viagens em "baratas", "médias", "caras" e "muito caras".

In [0]:
%sql
SELECT
  tpep_pickup_datetime, -- Incluído para contexto, mas NTILE opera sobre o conjunto ordenado
  fare_amount,
  NTILE(4) OVER (ORDER BY fare_amount) AS fare_amount_quartile
FROM
  trips_raw
WHERE
  fare_amount IS NOT NULL
ORDER BY
  fare_amount
LIMIT 50;

## duração da viagem, pode calculá-la a partir das colunas de data/hora: UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime) (resulta em segundos).

In [0]:
%sql
SELECT
pickup_zip,
tpep_pickup_datetime,
(UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime)) AS trip_duration_seconds,
LAG((UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime)), 1, 0) OVER (PARTITION BY pickup_zip ORDER BY tpep_pickup_datetime) AS previous_trip_duration_seconds
FROM
trips_raw
WHERE
pickup_zip IS NOT NULL
ORDER BY
pickup_zip,
tpep_pickup_datetime
LIMIT 20;